In [88]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
#http://scikit-learn.org/stable/modules/feature_selection.html

In [89]:
path='C:/Users/Lenovo/Desktop/iadap/3-2/IR/project/GOT-character-s-death-Prediction-master/'
data=pd.read_csv(path+'Formated data.csv')

x,y=list(),list()

In [90]:
def category(data):
    data.title=data.title.astype('category')  
    data.Culture=data.Culture.astype('category')        
    data.name=data.name.astype('category')
    data.house=data.house.astype('category')
    
    cat_columns = data.select_dtypes(['category']).columns
    data[cat_columns] = data[cat_columns].apply(lambda x: x.cat.codes)
    
    unique,counts=np.unique(data['title'],return_counts=True)
    diction=dict(zip(unique,counts))
    data['title']=data['title'].apply(lambda x: 1/diction[x]) 
    return data

In [91]:
def create_x_y(data):
    x=data.loc[:,'name':]
    #print(x)
    y=data.loc[:,'isAlive']  
    #print(y)
    return x,y

In [92]:
data=category(data)    
x,y=create_x_y(data)  
y=np.array(y)
y.shape

(1946,)

In [94]:
print(x.shape)
x_new = SelectKBest(chi2, k=20).fit_transform(x, y)
print(x_new.shape)

(1946, 54)
(1946, 20)


In [95]:
print(x_new[0])

[  1.82700000e+03   9.92063492e-04   1.00000000e+00   2.90000000e+01
   1.97000000e+02   0.00000000e+00   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00
   1.10000000e+01   6.05351171e-01   0.00000000e+00   0.00000000e+00
   0.00000000e+00   0.00000000e+00   0.00000000e+00   0.00000000e+00]


In [106]:
y_old=y
clf = ExtraTreesClassifier()
clf = clf.fit(x, y)
print(clf.feature_importances_ )

model = SelectFromModel(clf, prefit=True)
x_new = model.transform(x)
x_new.shape

y_new=np.zeros((len(y_old),2))
for i in range(len(y_old)):
    if y_old[i]==1:
        y_new[i]=[1,0]
        #print(y_new[i])
    else:
        y_new[i]=[0,1]
        #print(y_new[i])

[ 0.16374416  0.05062839  0.03581759  0.08384332  0.11227312  0.03025876
  0.02845147  0.02591006  0.07217657  0.03613253  0.02093533  0.02608473
  0.03903395  0.21772258  0.          0.0146708   0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.01945926
  0.02285737]


In [108]:
y_old

array([0, 1, 1, ..., 1, 1, 0], dtype=int64)

In [112]:
y=y_new.reshape((y_new.shape[0],2))
x_train,x_test,y_train,y_test=train_test_split(x_new,y,test_size=0.2)
print(y_train.shape)
print(x_train.shape)

(1556, 2)
(1556, 16)


In [113]:
random_state = 42
np.random.seed(random_state)
tf.set_random_seed(random_state)                

In [119]:
def multilayer_perceptron(x, weights, biases, keep_prob):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.nn.relu(layer_1)
    layer_1 = tf.nn.dropout(layer_1, keep_prob)
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    return out_layer

In [120]:
n_hidden_1 = 38
n_input = x_train.shape[1]
n_classes = 2

weights = {
    'h1': tf.Variable(tf.random_normal([n_input, n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_hidden_1, n_classes]))
}

biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

keep_prob = tf.placeholder("float")


In [124]:
training_epochs = 1000
display_step = 100
batch_size = 32

x = tf.placeholder("float", [None, n_input])
y = tf.placeholder("float", [None, n_classes])

In [125]:
predictions = multilayer_perceptron(x, weights, biases, keep_prob)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predictions, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.0001).minimize(cost)

In [126]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(training_epochs):
        avg_cost = 0.0
        total_batch = int(len(x_train) / batch_size)
        x_batches = np.array_split(x_train, total_batch)
        y_batches = np.array_split(y_train, total_batch)
        for i in range(total_batch):
            batch_x, batch_y = x_batches[i], y_batches[i]
            _, c = sess.run([optimizer, cost], 
                            feed_dict={
                                x: batch_x, 
                                y: batch_y, 
                                keep_prob: 0.8
                            })
            avg_cost += c / total_batch
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost))

    print("Optimization Finished!")

    correct_prediction = tf.equal(tf.argmax(predictions, 1), tf.argmax(y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Accuracy:", accuracy.eval({x: x_test, y: y_test, keep_prob: 1.0}))

Epoch: 0001 cost= 2497.564189911
Epoch: 0101 cost= 428.203108072
Epoch: 0201 cost= 179.933353821
Epoch: 0301 cost= 60.118754347
Epoch: 0401 cost= 3.754337029
Epoch: 0501 cost= 2.211734302
Epoch: 0601 cost= 1.571593535
Epoch: 0701 cost= 1.235603839
Epoch: 0801 cost= 1.259208518
Epoch: 0901 cost= 0.741661984
Optimization Finished!
Accuracy: 0.715385
